## Setup

In [1]:
import sys; sys.path.append('..')

In [2]:
# ! pip install lovely-tensors

import lovely_tensors as lt
lt.monkey_patch()

In [3]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader

from transformer_lens import HookedTransformer, HookedTransformerConfig

In [4]:
import wandb
from tqdm.auto import tqdm

from omegaconf import OmegaConf

from datetime import datetime

from pathlib import Path

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from src.tree import list2tree
from src.tree_dataset import TreeDataset, parse_input_idx, input_tokens_to_tree, tree_to_edges
from src.utils import seed_all


from src.trainer import accuracy_by_depth
from src.trainer import TreeTrainer

In [6]:
conf = OmegaConf.load('../conf/00_reproduce_6L_nodes=16.yaml')
# Output is identical to the YAML file
conf.n_nodes = 16
conf.device = 'cpu'
print(OmegaConf.to_yaml(conf))

random_seed: 42
n_nodes: 16
model:
  d_model: 128
  d_head: 128
  n_layers: 6
  act_fn: gelu
  attention_dir: causal
optimizer:
  lr: 0.001
  weight_decay: 0.01
batch_size: 64
epoch_len_steps: 5000
checkpoint_every_epoch: 2
device: cpu
debug: false
use_wandb: true
wandb:
  project: reasoning-mech-interp
  name: 00_6L_nodes=16
max_iters: null



In [7]:
REPRODUCED_MODEL_CKPT = '../checkpoints/reasoning-mech-interp__2024-04-10_16-10-20/00_6L_nodes=16__step=220000.pt'

In [8]:
device = conf.device

RANDOM_SEED = conf['random_seed']
print(f'{RANDOM_SEED=}')
seed_all(RANDOM_SEED)

RANDOM_SEED=42


In [9]:
trainer = TreeTrainer(conf)

tokenizer = trainer.dataset.tokenizer

tok = tokenizer.tokenize
detok = tokenizer.detokenize


ROOT_DELIM_TOKEN_IDX = trainer.tok([':'])[0]

state_dict = torch.load(REPRODUCED_MODEL_CKPT)
trainer.model.load_state_dict(state_dict)

Moving model to device:  cpu


<All keys matched successfully>

In [10]:
def load_baseline_model(device='cpu'):
    n_states = 16
    max_seq_length = n_states * 4 + 2
    
    number_tokens = sorted([str(i) for i in range(n_states)], key=lambda x: len(x), reverse=True)
    idx2tokens = [",", ":", "|"] + [f">{t}" for t in number_tokens] + number_tokens
    tokens2idx = {token: idx for idx, token in enumerate(idx2tokens)}
    
    
    cfg = HookedTransformerConfig(
        n_layers=6,
        d_model=128,
        n_ctx=max_seq_length - 1,
        n_heads=1,
        d_mlp=512,
        d_head=128,
        #attn_only=True,
        d_vocab=len(idx2tokens),
        device=device,
        attention_dir= "causal",
        act_fn="gelu",
    )
    model = HookedTransformer(cfg)
    
    model.load_state_dict(torch.load("/Users/mykhailokilianovskyi/src/backward-chaining-circuits/model.pt", map_location=torch.device(device)))
    
    return model

In [11]:
from src.tree_dataset import PAD_TOKEN

In [12]:
n_states = 16
bas_max_seq_length = n_states * 4 + 2

number_tokens = sorted([str(i) for i in range(n_states)], key=lambda x: len(x), reverse=True)
idx2tokens = [",", ":", "|"] + [f">{t}" for t in number_tokens] + number_tokens
tokens2idx = {token: idx for idx, token in enumerate(idx2tokens)}

In [13]:
token2bastoken = {k.replace('>', '→'):k for k in tokens2idx.keys()}
token2bastoken[PAD_TOKEN] = ','

our_idx2token = trainer.dataset.tokenizer.idx2token

idx2basidx = {}

In [14]:
for idx, our_tok in our_idx2token.items():
    bastoken = token2bastoken[our_tok]
    basidx = tokens2idx[bastoken]
    idx2basidx[idx] = basidx

In [15]:
def map_input_idx_to_bas(input_idx):
    return input_idx.clone().apply_(lambda i: idx2basidx[i])

## H#0: "Edge Embeddings" are learned

### Check that this info is in $x_1$ (residual stream after first layer)

In [16]:
seed_all(0)
for batch in trainer.dataloader: 
    break

In [17]:
self = trainer

In [18]:
input_idx = batch['input_idx'].to(self.device)
mask = batch['task_mask'].to(self.device)

inputs = input_idx[:, :-1]

out_mask = mask[:, 1:]
targets = input_idx[:, 1:][out_mask]


# print(input_idx[:1, :4])
outputs = self.model(inputs)

predictions = outputs[out_mask]

loss = F.cross_entropy(predictions, targets)

is_correct = (predictions.argmax(dim=-1) == targets)
accuracy_mean = is_correct.float().mean()
metrics = accuracy_by_depth(outputs, input_idx, out_mask)
metrics['accuracy/mean'] = accuracy_mean.item()

In [19]:
outputs, cache = self.model.run_with_cache(inputs)

In [20]:
l0_attention_pattern = cache['pattern', 0][0]
input_str = detok(inputs[0])

In [21]:
test_tree_tokens = detok(inputs[0])

In [22]:
trainer.print_sample_pred(inputs[0])

****************************************************************************************************
                            12
                             |
                    +--------+-----------+
                    |                    |
                   15                    2
                    |                    |
              +-----+-----+           +--+-----+
              |           |           |        |
              4           6           5        9
              |           |           |        |
     +--------+--+     +--+        +--+     +--+
     |           |     |           |        |
     8           3     0          14        1
     |                             |
  +--+-----+                    +--+
  |        |                    |
 11        7                   10
           |
        +--+
        |
       13

goal=3
accuracy=1.0 gt_path=['→15', '→4', '→3'] pred_path=['→15', '→4', '→3']
*************************************************************

In [23]:
import circuitsvis as cv

In [24]:
print("Layer 0 Head Attention Pattern:")
display(cv.attention.attention_patterns(
    tokens=input_str,
    attention=l0_attention_pattern,
    attention_head_names=[f"L0H{i}" for i in range(1)],
))

Layer 0 Head Attention Pattern:


#### Observation: last edge is special

 - looks like "edge embeddings" are accumulated in pad token `,`
 - but there is no `,` for the last edge (both in my impl and baseline impl)
 - attn pattern looks weird there
 - what if our path goes through last edge? Will performance drop?

In [25]:
from copy import deepcopy

In [26]:
def swap_edges(seq, e0, e1):
    seq = deepcopy(seq)
    
    e0i = -1
    e1i = -1
    
    for i in range(len(seq)-1):
        if tuple(seq[i:i+2]) == e0: e0i = i
    
    for i in range(len(seq)-1):
        if tuple(seq[i:i+2]) == e1: e1i = i
    
    seq[e0i:e0i+2] = e1
    seq[e1i:e1i+2] = e0
    return seq

In [27]:
# swap edge (12→15) with last edge (14→10)

e0 = ('15', '→4')
e1 = ('14', '→10')
input_str_swap0 = swap_edges(input_str, e0, e1)[:68]
input_idx_swap0 = torch.tensor(tok(input_str_swap0))

outputs, cache = self.model.run_with_cache(input_idx_swap0[None])

In [28]:
# for x,y in zip(input_str_swap0, input_str):
#     print(x,y)

In [29]:
input_str_swap0 == input_str

False

In [30]:
l0_attention_pattern

tensor[1, 67, 67] n=4489 (18Kb) x∈[0., 1.000] μ=0.015 σ=0.045

In [31]:
len(input_str_swap0)

67

In [32]:
print("Layer 0 Head Attention Pattern:")
l0_attention_pattern = cache['pattern', 0][0]
display(cv.attention.attention_patterns(
    tokens=input_str_swap0,
    attention=l0_attention_pattern,
    attention_head_names=[f"L0H{i}" for i in range(1)],
))

Layer 0 Head Attention Pattern:


In [33]:
trainer.print_sample_pred(input_idx[0])

****************************************************************************************************
                            12
                             |
                    +--------+-----------+
                    |                    |
                   15                    2
                    |                    |
              +-----+-----+           +--+-----+
              |           |           |        |
              4           6           5        9
              |           |           |        |
     +--------+--+     +--+        +--+     +--+
     |           |     |           |        |
     8           3     0          14        1
     |                             |
  +--+-----+                    +--+
  |        |                    |
 11        7                   10
           |
        +--+
        |
       13

goal=3
accuracy=1.0 gt_path=['→15', '→4', '→3'] pred_path=['→15', '→4', '→3']
*************************************************************

In [34]:
trainer.print_sample_pred(input_idx_swap0)

****************************************************************************************************
                            12
                             |
        +--------------------+-----------+
        |                                |
       15                                2
        |                                |
     +--+--------------+              +--+-----+
     |                 |              |        |
     6                 4              5        9
     |                 |              |        |
  +--+        +--------+--+        +--+     +--+
  |           |           |        |        |
  0           8           3       14        1
              |                    |
           +--+-----+           +--+
           |        |           |
          11        7          10
                    |
                 +--+
                 |
                13

goal=3
accuracy=1.0 gt_path=['→15', '→4', '→3'] pred_path=['→15', '→4', '→3']
*************************

#### is it also special in baseline model?

In [35]:
baseline_model = load_baseline_model()

In [36]:
l0_attention_pattern.squeeze()

tensor[67, 67] n=4489 (18Kb) x∈[0., 1.000] μ=0.015 σ=0.045

In [37]:
bas_input_idx = map_input_idx_to_bas(input_idx[0])[:bas_max_seq_length-1]
bas_input_str = [idx2tokens[i] for i in bas_input_idx]

outputs, cache = baseline_model.run_with_cache(bas_input_idx[None])
l0_attention_pattern_bas = cache['pattern', 0][0]

print("Baseline L0 Head Attention Pattern:")

display(cv.attention.attention_patterns(
    tokens=bas_input_str,
    attention=l0_attention_pattern_bas,
    attention_head_names=[f"L0H{i}" for i in range(1)],
))

Baseline L0 Head Attention Pattern:


In [38]:
bas_input_idx = map_input_idx_to_bas(input_idx_swap0)[:bas_max_seq_length-1]
bas_input_str = [idx2tokens[i] for i in bas_input_idx]

outputs, cache = baseline_model.run_with_cache(bas_input_idx[None])
l0_attention_pattern_bas = cache['pattern', 0][0]

print("Baseline L0 Head Attention Pattern:")

display(cv.attention.attention_patterns(
    tokens=bas_input_str,
    attention=l0_attention_pattern_bas,
    attention_head_names=[f"L0H{i}" for i in range(1)],
))

Baseline L0 Head Attention Pattern:


In [39]:
print("Layer 1 Head Attention Pattern:")

l1_attention_pattern_bas = cache['pattern', 1][0]

display(cv.attention.attention_patterns(
    tokens=bas_input_str,
    attention=l1_attention_pattern_bas,
    attention_head_names=[f"L1H{i}" for i in range(1)],
))

Layer 1 Head Attention Pattern:


### Do so called Linear Probe

How to do linear probe?

  - Take info from the unit of interest and try to classify it.

Paper makes the hypothesis that first layer aggregates info about edge into the target pos.

> the model aggregates the source and target nodes of each edge in the edge list into the target node position

  - [ ] How to access residual stream in TransformerLens?


> cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type].


`final_residual_stream = cache["resid_post", -1]`

In [40]:
from src.tree_dataset import TreeDataset, DeepTreeDataset, parse_input_idx, input_tokens_to_tree, tree_to_edges

test_dataloader = DataLoader ( DeepTreeDataset(possible_depths=list(range(1,16))), batch_size=64 )

In [41]:
for batch in test_dataloader:
    break

In [42]:
model = trainer.model

In [43]:
torch.inference_mode()
def get_cache(batch, model):
    input_idx = batch['input_idx'].to(device)
    mask = batch['task_mask'].to(device)

    inputs = input_idx[:, :-1]
    
    out_mask = mask[:, 1:]
    targets = input_idx[:, 1:][out_mask]
    
    outputs, cache = model.run_with_cache(inputs)
    return cache

In [44]:
# we expect second node in edge to contain info about two nodes in edge

In [45]:
sample_input_idx = input_idx[8]

ROOT_DELIM_TOKEN_IDX = trainer.tok([':'])[0]

def extract_edges(sample_input_idx):
    upper_task_bound = sample_input_idx.tolist().index(ROOT_DELIM_TOKEN_IDX) + 2
    prompt = trainer.detok(sample_input_idx)[:upper_task_bound]
    
    i = 2
    
    edges = []
    
    while i < len(prompt) and prompt[i] != '|':
        edge = (prompt[i-2], prompt[i-1])
        edges.append(edge)
        i += 3
    
    return edges

In [46]:
n_nodes = 16

In [47]:
edges = [(str(i), '→'+str(j)) for i in range(n_nodes) for j in range(n_nodes)]

edge2idx = {e:i for i,e in enumerate(edges)}

idx2edge = {i:e for e,i in edge2idx.items()}
len(edge2idx)

256

In [49]:


def get_first_node(edge_idx): return int(idx2edge[edge_idx][0])
def get_second_node(edge_idx): return int(idx2edge[edge_idx][1][1:])

In [50]:
def get_edge_labels(batch):
    input_idx = batch['input_idx']
    edge_batch = []
    for row in input_idx:
        edges = extract_edges(row)
        edges = [edge2idx[e] for e in edges]
        edge_batch.append(edges)
    return torch.tensor(edge_batch)

In [51]:
second_token_idx = torch.arange(1, 14*3, 3)
second_token_idx.v

tensor[14] i64 x∈[1, 40] μ=20.500 σ=12.550
tensor([ 1,  4,  7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40])

In [52]:
d_model = conf.model.d_model

In [53]:
def extract_Ln_second_node_xy(batch, model, key='resid_post', n=0):
    cache = get_cache(batch, model)
    resid_act0 = cache[key, n]
    
    X = resid_act0[:, second_token_idx].reshape(-1, d_model)
    y = get_edge_labels(batch).reshape(-1)
    
    return X,y

In [54]:
model = trainer.model

In [55]:
extract_Ln_second_node_xy(batch, model)

(tensor[896, 128] n=114688 (0.4Mb) x∈[-19.844, 12.516] μ=-0.014 σ=3.187,
 tensor[896] i64 7Kb x∈[1, 254] μ=126.410 σ=72.792)

In [56]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
from torch import nn


class LogisticRegressionModel(nn.Module):
    def __init__(self, n_features, n_classes):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(n_features, n_classes)
    
    def forward(self, x):
        return self.linear(x)

# Assuming n_features is the number of features in your dataset and n_classes is the number of classes


In [59]:
num_epochs = 100

In [60]:
from torch.utils.data import DataLoader, TensorDataset

def run_logreg(X,y, num_epochs=5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    logreg = LogisticRegressionModel(X_train.shape[1], len(np.unique(y_train)))
    
    logreg_train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(logreg_train_dataset, batch_size=64, shuffle=True)
    
    logreg_test_dataset = TensorDataset(X_test, y_test)
    logreg_test_loader = DataLoader(logreg_test_dataset, batch_size=64, shuffle=True)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(logreg.parameters(), lr=0.001)
    
    
    train_losses = []
    train_accuracies = []
    
    test_losses = []
    test_accuracies = []
    
    for epoch in range(num_epochs):
        logreg.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = logreg(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
    
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
    
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in logreg_test_loader:
    
            with torch.inference_mode():
                outputs = logreg(inputs)
    
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        epoch_loss = running_loss / len(logreg_test_loader)
        epoch_accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)
    
        print(f'Test Loss: {epoch_loss:.4f}, Test Accuracy: {epoch_accuracy:.2f}%')
    return epoch_accuracy

### Linear Probe x_1

In [62]:
# n_batches = 100
# X = []
# y = []

# for i,batch in tqdm(zip(range(n_batches), test_dataloader), total=n_batches):
#     Xbatch, ybatch = extract_Ln_second_node_xy(batch, model, n=0)

#     X.append(Xbatch)
#     y.append(ybatch)


# X = torch.cat(X)
# y = torch.cat(y)

# y_first_node = torch.tensor( [get_first_node(i.item()) for i in y] )
# y_second_node = torch.tensor( [get_second_node(i.item()) for i in y] )

In [63]:
# run_logreg(X, y_first_node)

In [64]:
# run_logreg(X, y_second_node)

### Linear Probe x_0

In [65]:
# cache['embed']

In [66]:


# n_batches = 100
# X = []
# y = []

# for i,batch in tqdm(zip(range(n_batches), test_dataloader), total=n_batches):
#     Xbatch, ybatch = extract_Ln_second_node_xy(batch, model,key='embed', n=None)

#     X.append(Xbatch)
#     y.append(ybatch)


# X = torch.cat(X)
# y = torch.cat(y)

# y_first_node = torch.tensor( [get_first_node(i.item()) for i in y] )
# y_second_node = torch.tensor( [get_second_node(i.item()) for i in y] )

In [67]:
# run_logreg(X, y_first_node)

In [68]:
# run_logreg(X, y_second_node)

### Linear Probe each layer!

In [69]:
cache_keys = [{'key': 'resid_post', "n": n} for n in range(6)]
cache_keys = [{'key': 'embed', 'n': None}] + cache_keys
cache_keys

[{'key': 'embed', 'n': None},
 {'key': 'resid_post', 'n': 0},
 {'key': 'resid_post', 'n': 1},
 {'key': 'resid_post', 'n': 2},
 {'key': 'resid_post', 'n': 3},
 {'key': 'resid_post', 'n': 4},
 {'key': 'resid_post', 'n': 5}]

In [70]:
accuracies = []

for cache_key in cache_keys:
    n_batches = 100
    X = []
    y = []
    
    for i,batch in tqdm(zip(range(n_batches), test_dataloader), total=n_batches):
        Xbatch, ybatch = extract_Ln_second_node_xy(batch, model, **cache_key)
    
        X.append(Xbatch)
        y.append(ybatch)
    
    
    X = torch.cat(X)
    y = torch.cat(y)
    
    y_first_node = torch.tensor( [get_first_node(i.item()) for i in y] )
    y_second_node = torch.tensor( [get_second_node(i.item()) for i in y] )

    acc_node0 = run_logreg(X, y_first_node)
    acc_node1 = run_logreg(X, y_second_node)


    layer_name = ''.join ( [f'{k}={v} ' for k,v in cache_key.items()] )[:-1]
    metric = {'acc_node_0': float(acc_node0), 'acc_node_1': float(acc_node1), 'layer_name':layer_name}
    print(f'{metric=}')

    accuracies.append(metric)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.7372, Accuracy: 6.57%
Test Loss: 0.0000, Test Accuracy: 6.66%
Epoch [2/5], Loss: 2.7200, Accuracy: 6.98%
Test Loss: 0.0000, Test Accuracy: 6.18%
Epoch [3/5], Loss: 2.7171, Accuracy: 6.77%
Test Loss: 0.0000, Test Accuracy: 6.64%
Epoch [4/5], Loss: 2.7154, Accuracy: 6.71%
Test Loss: 0.0000, Test Accuracy: 6.47%
Epoch [5/5], Loss: 2.7146, Accuracy: 6.69%
Test Loss: 0.0000, Test Accuracy: 6.81%
Epoch [1/5], Loss: 0.2823, Accuracy: 98.83%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [2/5], Loss: 0.0091, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [3/5], Loss: 0.0032, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [4/5], Loss: 0.0015, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [5/5], Loss: 0.0008, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
metric={'acc_node_0': 6.808035714285714, 'acc_node_1': 100.0, 'layer_name': 'key=embed n=None'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 0.2165, Accuracy: 93.33%
Test Loss: 0.0000, Test Accuracy: 95.78%
Epoch [2/5], Loss: 0.0903, Accuracy: 97.49%
Test Loss: 0.0000, Test Accuracy: 98.73%
Epoch [3/5], Loss: 0.0516, Accuracy: 99.13%
Test Loss: 0.0000, Test Accuracy: 99.58%
Epoch [4/5], Loss: 0.0303, Accuracy: 99.67%
Test Loss: 0.0000, Test Accuracy: 99.80%
Epoch [5/5], Loss: 0.0190, Accuracy: 99.86%
Test Loss: 0.0000, Test Accuracy: 99.98%
Epoch [1/5], Loss: 0.2699, Accuracy: 95.17%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [2/5], Loss: 0.0133, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [3/5], Loss: 0.0042, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [4/5], Loss: 0.0020, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
Epoch [5/5], Loss: 0.0010, Accuracy: 100.00%
Test Loss: 0.0000, Test Accuracy: 100.00%
metric={'acc_node_0': 99.97767857142857, 'acc_node_1': 100.0, 'layer_name': 'key=resid_post n=0'}


  0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch [1/5], Loss: 2.7992, Accuracy: 74.32%
Test Loss: 0.0000, Test Accuracy: 90.37%
Epoch [2/5], Loss: 0.3392, Accuracy: 91.32%
Test Loss: 0.0000, Test Accuracy: 92.02%
Epoch [3/5], Loss: 0.2821, Accuracy: 92.31%
Test Loss: 0.0000, Test Accuracy: 93.11%
Epoch [4/5], Loss: 0.2635, Accuracy: 92.61%
Test Loss: 0.0000, Test Accuracy: 93.21%
Epoch [5/5], Loss: 0.2518, Accuracy: 92.94%
Test Loss: 0.0000, Test Accuracy: 93.62%
Epoch [1/5], Loss: 2.5386, Accuracy: 74.95%
Test Loss: 0.0000, Test Accuracy: 89.86%
Epoch [2/5], Loss: 0.3814, Accuracy: 91.02%
Test Loss: 0.0000, Test Accuracy: 93.60%
Epoch [3/5], Loss: 0.2660, Accuracy: 93.17%
Test Loss: 0.0000, Test Accuracy: 94.92%
Epoch [4/5], Loss: 0.2005, Accuracy: 94.72%
Test Loss: 0.0000, Test Accuracy: 95.86%
Epoch [5/5], Loss: 0.1556, Accuracy: 96.01%
Test Loss: 0.0000, Test Accuracy: 95.36%
metric={'acc_node_0': 93.62165178571429, 'acc_node_1': 95.35714285714286, 'layer_name': 'key=resid_post n=3'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.6661, Accuracy: 73.46%
Test Loss: 0.0000, Test Accuracy: 90.61%
Epoch [2/5], Loss: 0.3337, Accuracy: 91.20%
Test Loss: 0.0000, Test Accuracy: 92.81%
Epoch [3/5], Loss: 0.2849, Accuracy: 92.25%
Test Loss: 0.0000, Test Accuracy: 92.38%
Epoch [4/5], Loss: 0.2686, Accuracy: 92.57%
Test Loss: 0.0000, Test Accuracy: 93.74%
Epoch [5/5], Loss: 0.2443, Accuracy: 93.13%
Test Loss: 0.0000, Test Accuracy: 93.54%
Epoch [1/5], Loss: 2.5672, Accuracy: 72.75%
Test Loss: 0.0000, Test Accuracy: 89.55%
Epoch [2/5], Loss: 0.3903, Accuracy: 90.65%
Test Loss: 0.0000, Test Accuracy: 89.47%
Epoch [3/5], Loss: 0.2665, Accuracy: 93.23%
Test Loss: 0.0000, Test Accuracy: 95.35%
Epoch [4/5], Loss: 0.1910, Accuracy: 95.11%
Test Loss: 0.0000, Test Accuracy: 96.53%
Epoch [5/5], Loss: 0.1484, Accuracy: 96.26%
Test Loss: 0.0000, Test Accuracy: 96.90%
metric={'acc_node_0': 93.54352678571429, 'acc_node_1': 96.89732142857143, 'layer_name': 'key=resid_post n=4'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.1108, Accuracy: 70.69%
Test Loss: 0.0000, Test Accuracy: 86.80%
Epoch [2/5], Loss: 0.3549, Accuracy: 90.92%
Test Loss: 0.0000, Test Accuracy: 92.14%
Epoch [3/5], Loss: 0.2898, Accuracy: 92.25%
Test Loss: 0.0000, Test Accuracy: 93.06%
Epoch [4/5], Loss: 0.2572, Accuracy: 92.77%
Test Loss: 0.0000, Test Accuracy: 93.23%
Epoch [5/5], Loss: 0.2357, Accuracy: 93.21%
Test Loss: 0.0000, Test Accuracy: 91.93%
Epoch [1/5], Loss: 2.7860, Accuracy: 71.25%
Test Loss: 0.0000, Test Accuracy: 88.64%
Epoch [2/5], Loss: 0.3984, Accuracy: 90.08%
Test Loss: 0.0000, Test Accuracy: 89.34%
Epoch [3/5], Loss: 0.2673, Accuracy: 92.78%
Test Loss: 0.0000, Test Accuracy: 93.09%
Epoch [4/5], Loss: 0.1956, Accuracy: 94.96%
Test Loss: 0.0000, Test Accuracy: 96.09%
Epoch [5/5], Loss: 0.1412, Accuracy: 96.56%
Test Loss: 0.0000, Test Accuracy: 97.00%
metric={'acc_node_0': 91.93080357142857, 'acc_node_1': 96.99776785714286, 'layer_name': 'key=resid_post n=5'}


In [71]:
pd.DataFrame(accuracies)

,acc_node_0,acc_node_1,layer_name
0,6.808036,100.000000,key=embed n=None
1,99.977679,100.000000,key=resid_post n=0
2,96.021205,99.921875,key=resid_post n=1
3,94.648438,99.174107,key=resid_post n=2
4,93.621652,95.357143,key=resid_post n=3
5,93.543527,96.897321,key=resid_post n=4
6,91.930804,96.997768,key=resid_post n=5


### probe token that is next to second node token

In [72]:
third_token_idx = torch.arange(2, 14*3, 3)
third_token_idx.v

tensor[14] i64 x∈[2, 41] μ=21.500 σ=12.550
tensor([ 2,  5,  8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41])

In [73]:
def extract_Ln_third_node(batch, model, key='resid_post', n=0):
    cache = get_cache(batch, model)
    resid_act0 = cache[key, n]
    
    X = resid_act0[:, third_token_idx].reshape(-1, d_model)
    y = get_edge_labels(batch).reshape(-1)
    
    return X,y

In [74]:
accuracies = []

for cache_key in cache_keys:
    n_batches = 100
    X = []
    y = []
    
    for i,batch in tqdm(zip(range(n_batches), test_dataloader), total=n_batches):
        Xbatch, ybatch = extract_Ln_third_node(batch, model, **cache_key)
    
        X.append(Xbatch)
        y.append(ybatch)
    
    
    X = torch.cat(X)
    y = torch.cat(y)
    
    y_first_node = torch.tensor( [get_first_node(i.item()) for i in y] )
    y_second_node = torch.tensor( [get_second_node(i.item()) for i in y] )

    acc_node0 = run_logreg(X, y_first_node)
    acc_node1 = run_logreg(X, y_second_node)


    layer_name = ''.join ( [f'{k}={v} ' for k,v in cache_key.items()] )[:-1]
    metric = {'acc_node_0': float(acc_node0), 'acc_node_1': float(acc_node1), 'layer_name':layer_name}
    print(f'{metric=}')

    accuracies.append(metric)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.7750, Accuracy: 6.22%
Test Loss: 0.0000, Test Accuracy: 6.19%
Epoch [2/5], Loss: 2.7747, Accuracy: 6.16%
Test Loss: 0.0000, Test Accuracy: 6.36%
Epoch [3/5], Loss: 2.7749, Accuracy: 6.27%
Test Loss: 0.0000, Test Accuracy: 6.29%
Epoch [4/5], Loss: 2.7746, Accuracy: 6.26%
Test Loss: 0.0000, Test Accuracy: 6.36%
Epoch [5/5], Loss: 2.7752, Accuracy: 6.20%
Test Loss: 0.0000, Test Accuracy: 6.06%
Epoch [1/5], Loss: 2.7751, Accuracy: 6.26%
Test Loss: 0.0000, Test Accuracy: 6.34%
Epoch [2/5], Loss: 2.7749, Accuracy: 6.14%
Test Loss: 0.0000, Test Accuracy: 6.16%
Epoch [3/5], Loss: 2.7748, Accuracy: 6.24%
Test Loss: 0.0000, Test Accuracy: 6.24%
Epoch [4/5], Loss: 2.7751, Accuracy: 6.10%
Test Loss: 0.0000, Test Accuracy: 6.12%
Epoch [5/5], Loss: 2.7749, Accuracy: 6.23%
Test Loss: 0.0000, Test Accuracy: 6.34%
metric={'acc_node_0': 6.060267857142857, 'acc_node_1': 6.339285714285714, 'layer_name': 'key=embed n=None'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 0.2538, Accuracy: 92.61%
Test Loss: 0.0000, Test Accuracy: 93.62%
Epoch [2/5], Loss: 0.2025, Accuracy: 93.51%
Test Loss: 0.0000, Test Accuracy: 93.65%
Epoch [3/5], Loss: 0.2019, Accuracy: 93.58%
Test Loss: 0.0000, Test Accuracy: 93.68%
Epoch [4/5], Loss: 0.2026, Accuracy: 93.51%
Test Loss: 0.0000, Test Accuracy: 93.64%
Epoch [5/5], Loss: 0.2008, Accuracy: 93.56%
Test Loss: 0.0000, Test Accuracy: 93.60%
Epoch [1/5], Loss: 0.3663, Accuracy: 91.44%
Test Loss: 0.0000, Test Accuracy: 93.30%
Epoch [2/5], Loss: 0.2753, Accuracy: 93.25%
Test Loss: 0.0000, Test Accuracy: 93.31%
Epoch [3/5], Loss: 0.2686, Accuracy: 93.23%
Test Loss: 0.0000, Test Accuracy: 93.34%
Epoch [4/5], Loss: 0.2624, Accuracy: 93.25%
Test Loss: 0.0000, Test Accuracy: 93.28%
Epoch [5/5], Loss: 0.2568, Accuracy: 93.27%
Test Loss: 0.0000, Test Accuracy: 93.34%
metric={'acc_node_0': 93.59933035714286, 'acc_node_1': 93.33705357142857, 'layer_name': 'key=resid_post n=0'}


  0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch [1/5], Loss: 1.0027, Accuracy: 90.09%
Test Loss: 0.0000, Test Accuracy: 93.35%
Epoch [2/5], Loss: 0.5184, Accuracy: 93.34%
Test Loss: 0.0000, Test Accuracy: 93.51%
Epoch [3/5], Loss: 0.5192, Accuracy: 93.31%
Test Loss: 0.0000, Test Accuracy: 93.52%
Epoch [4/5], Loss: 0.4885, Accuracy: 93.34%
Test Loss: 0.0000, Test Accuracy: 93.44%
Epoch [5/5], Loss: 0.4985, Accuracy: 93.41%
Test Loss: 0.0000, Test Accuracy: 93.52%
Epoch [1/5], Loss: 0.8446, Accuracy: 90.72%
Test Loss: 0.0000, Test Accuracy: 93.23%
Epoch [2/5], Loss: 0.5637, Accuracy: 93.21%
Test Loss: 0.0000, Test Accuracy: 93.23%
Epoch [3/5], Loss: 0.5260, Accuracy: 93.25%
Test Loss: 0.0000, Test Accuracy: 93.40%
Epoch [4/5], Loss: 0.5152, Accuracy: 93.30%
Test Loss: 0.0000, Test Accuracy: 93.28%
Epoch [5/5], Loss: 0.5191, Accuracy: 93.30%
Test Loss: 0.0000, Test Accuracy: 93.44%
metric={'acc_node_0': 93.515625, 'acc_node_1': 93.4375, 'layer_name': 'key=resid_post n=3'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 1.0941, Accuracy: 89.39%
Test Loss: 0.0000, Test Accuracy: 93.20%
Epoch [2/5], Loss: 0.5441, Accuracy: 93.26%
Test Loss: 0.0000, Test Accuracy: 93.47%
Epoch [3/5], Loss: 0.5012, Accuracy: 93.30%
Test Loss: 0.0000, Test Accuracy: 93.33%
Epoch [4/5], Loss: 0.4934, Accuracy: 93.35%
Test Loss: 0.0000, Test Accuracy: 93.57%
Epoch [5/5], Loss: 0.4992, Accuracy: 93.37%
Test Loss: 0.0000, Test Accuracy: 93.41%
Epoch [1/5], Loss: 1.0576, Accuracy: 90.20%
Test Loss: 0.0000, Test Accuracy: 93.26%
Epoch [2/5], Loss: 0.5664, Accuracy: 93.17%
Test Loss: 0.0000, Test Accuracy: 93.18%
Epoch [3/5], Loss: 0.5670, Accuracy: 93.20%
Test Loss: 0.0000, Test Accuracy: 93.26%
Epoch [4/5], Loss: 0.5424, Accuracy: 93.26%
Test Loss: 0.0000, Test Accuracy: 93.28%
Epoch [5/5], Loss: 0.5528, Accuracy: 93.29%
Test Loss: 0.0000, Test Accuracy: 93.35%
metric={'acc_node_0': 93.40959821428571, 'acc_node_1': 93.35379464285714, 'layer_name': 'key=resid_post n=4'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 1.0476, Accuracy: 89.27%
Test Loss: 0.0000, Test Accuracy: 93.20%
Epoch [2/5], Loss: 0.5509, Accuracy: 93.19%
Test Loss: 0.0000, Test Accuracy: 93.26%
Epoch [3/5], Loss: 0.5014, Accuracy: 93.29%
Test Loss: 0.0000, Test Accuracy: 93.41%
Epoch [4/5], Loss: 0.5022, Accuracy: 93.31%
Test Loss: 0.0000, Test Accuracy: 93.34%
Epoch [5/5], Loss: 0.4955, Accuracy: 93.38%
Test Loss: 0.0000, Test Accuracy: 93.33%
Epoch [1/5], Loss: 0.9927, Accuracy: 89.73%
Test Loss: 0.0000, Test Accuracy: 93.28%
Epoch [2/5], Loss: 0.5733, Accuracy: 93.21%
Test Loss: 0.0000, Test Accuracy: 93.33%
Epoch [3/5], Loss: 0.5685, Accuracy: 93.24%
Test Loss: 0.0000, Test Accuracy: 93.26%
Epoch [4/5], Loss: 0.5511, Accuracy: 93.30%
Test Loss: 0.0000, Test Accuracy: 93.33%
Epoch [5/5], Loss: 0.5268, Accuracy: 93.26%
Test Loss: 0.0000, Test Accuracy: 93.43%
metric={'acc_node_0': 93.33147321428571, 'acc_node_1': 93.43191964285714, 'layer_name': 'key=resid_post n=5'}


In [77]:
pd.DataFrame(accuracies)

,acc_node_0,acc_node_1,layer_name
0,6.060268,6.339286,key=embed n=None
1,93.599330,93.337054,key=resid_post n=0
2,93.448661,93.448661,key=resid_post n=1
3,93.515625,93.242188,key=resid_post n=2
4,93.515625,93.437500,key=resid_post n=3
5,93.409598,93.353795,key=resid_post n=4
6,93.331473,93.431920,key=resid_post n=5


In [80]:
fourth_token_idx = torch.arange(3, 15*3, 3) # <<-- Nota Bene magic number change
fourth_token_idx.v

tensor[14] i64 x∈[3, 42] μ=22.500 σ=12.550
tensor([ 3,  6,  9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42])

In [81]:
def extract_Ln_fourth_node(batch, model, key='resid_post', n=0):
    cache = get_cache(batch, model)
    resid_act0 = cache[key, n]
    
    X = resid_act0[:, fourth_token_idx].reshape(-1, d_model)
    y = get_edge_labels(batch).reshape(-1)
    
    return X,y

In [82]:
accuracies = []

for cache_key in cache_keys:
    n_batches = 100
    X = []
    y = []
    
    for i,batch in tqdm(zip(range(n_batches), test_dataloader), total=n_batches):
        Xbatch, ybatch = extract_Ln_fourth_node(batch, model, **cache_key)
    
        X.append(Xbatch)
        y.append(ybatch)
    
    
    X = torch.cat(X)
    y = torch.cat(y)
    
    y_first_node = torch.tensor( [get_first_node(i.item()) for i in y] )
    y_second_node = torch.tensor( [get_second_node(i.item()) for i in y] )

    acc_node0 = run_logreg(X, y_first_node)
    acc_node1 = run_logreg(X, y_second_node)


    layer_name = ''.join ( [f'{k}={v} ' for k,v in cache_key.items()] )[:-1]
    metric = {'acc_node_0': float(acc_node0), 'acc_node_1': float(acc_node1), 'layer_name':layer_name}
    print(f'{metric=}')

    accuracies.append(metric)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.7699, Accuracy: 6.55%
Test Loss: 0.0000, Test Accuracy: 6.38%
Epoch [2/5], Loss: 2.7669, Accuracy: 6.63%
Test Loss: 0.0000, Test Accuracy: 5.94%
Epoch [3/5], Loss: 2.7667, Accuracy: 6.71%
Test Loss: 0.0000, Test Accuracy: 6.35%
Epoch [4/5], Loss: 2.7670, Accuracy: 6.86%
Test Loss: 0.0000, Test Accuracy: 6.43%
Epoch [5/5], Loss: 2.7669, Accuracy: 6.81%
Test Loss: 0.0000, Test Accuracy: 6.29%
Epoch [1/5], Loss: 2.7761, Accuracy: 6.22%
Test Loss: 0.0000, Test Accuracy: 6.24%
Epoch [2/5], Loss: 2.7749, Accuracy: 6.52%
Test Loss: 0.0000, Test Accuracy: 6.41%
Epoch [3/5], Loss: 2.7749, Accuracy: 6.44%
Test Loss: 0.0000, Test Accuracy: 6.26%
Epoch [4/5], Loss: 2.7751, Accuracy: 6.44%
Test Loss: 0.0000, Test Accuracy: 6.24%
Epoch [5/5], Loss: 2.7748, Accuracy: 6.47%
Test Loss: 0.0000, Test Accuracy: 6.23%
metric={'acc_node_0': 6.2890625, 'acc_node_1': 6.227678571428571, 'layer_name': 'key=embed n=None'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.7000, Accuracy: 11.90%
Test Loss: 0.0000, Test Accuracy: 13.24%
Epoch [2/5], Loss: 2.5851, Accuracy: 14.14%
Test Loss: 0.0000, Test Accuracy: 14.30%
Epoch [3/5], Loss: 2.5643, Accuracy: 15.12%
Test Loss: 0.0000, Test Accuracy: 15.28%
Epoch [4/5], Loss: 2.5523, Accuracy: 15.39%
Test Loss: 0.0000, Test Accuracy: 15.39%
Epoch [5/5], Loss: 2.5452, Accuracy: 15.75%
Test Loss: 0.0000, Test Accuracy: 15.62%
Epoch [1/5], Loss: 2.8052, Accuracy: 10.69%
Test Loss: 0.0000, Test Accuracy: 11.54%
Epoch [2/5], Loss: 2.7134, Accuracy: 11.66%
Test Loss: 0.0000, Test Accuracy: 11.75%
Epoch [3/5], Loss: 2.6988, Accuracy: 11.89%
Test Loss: 0.0000, Test Accuracy: 11.71%
Epoch [4/5], Loss: 2.6878, Accuracy: 11.97%
Test Loss: 0.0000, Test Accuracy: 12.17%
Epoch [5/5], Loss: 2.6843, Accuracy: 12.00%
Test Loss: 0.0000, Test Accuracy: 12.14%
metric={'acc_node_0': 15.625, 'acc_node_1': 12.142857142857142, 'layer_name': 'key=resid_post n=0'}


  0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch [1/5], Loss: 8.7660, Accuracy: 11.95%
Test Loss: 0.0000, Test Accuracy: 15.85%
Epoch [2/5], Loss: 3.1671, Accuracy: 18.27%
Test Loss: 0.0000, Test Accuracy: 20.40%
Epoch [3/5], Loss: 3.0330, Accuracy: 20.98%
Test Loss: 0.0000, Test Accuracy: 20.20%
Epoch [4/5], Loss: 2.9593, Accuracy: 22.32%
Test Loss: 0.0000, Test Accuracy: 24.53%
Epoch [5/5], Loss: 2.8357, Accuracy: 23.64%
Test Loss: 0.0000, Test Accuracy: 26.86%
Epoch [1/5], Loss: 8.1266, Accuracy: 10.68%
Test Loss: 0.0000, Test Accuracy: 14.25%
Epoch [2/5], Loss: 3.3946, Accuracy: 15.78%
Test Loss: 0.0000, Test Accuracy: 17.63%
Epoch [3/5], Loss: 3.2460, Accuracy: 17.27%
Test Loss: 0.0000, Test Accuracy: 19.43%
Epoch [4/5], Loss: 3.2078, Accuracy: 18.09%
Test Loss: 0.0000, Test Accuracy: 21.07%
Epoch [5/5], Loss: 3.1144, Accuracy: 19.15%
Test Loss: 0.0000, Test Accuracy: 20.03%
metric={'acc_node_0': 26.858258928571427, 'acc_node_1': 20.033482142857142, 'layer_name': 'key=resid_post n=3'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 6.6156, Accuracy: 11.97%
Test Loss: 0.0000, Test Accuracy: 13.54%
Epoch [2/5], Loss: 3.1955, Accuracy: 17.86%
Test Loss: 0.0000, Test Accuracy: 19.93%
Epoch [3/5], Loss: 3.0218, Accuracy: 20.42%
Test Loss: 0.0000, Test Accuracy: 21.32%
Epoch [4/5], Loss: 3.0546, Accuracy: 21.42%
Test Loss: 0.0000, Test Accuracy: 25.86%
Epoch [5/5], Loss: 2.8633, Accuracy: 22.75%
Test Loss: 0.0000, Test Accuracy: 25.42%
Epoch [1/5], Loss: 8.3160, Accuracy: 10.25%
Test Loss: 0.0000, Test Accuracy: 13.91%
Epoch [2/5], Loss: 3.3692, Accuracy: 15.84%
Test Loss: 0.0000, Test Accuracy: 15.48%
Epoch [3/5], Loss: 3.2306, Accuracy: 17.37%
Test Loss: 0.0000, Test Accuracy: 19.33%
Epoch [4/5], Loss: 3.1600, Accuracy: 18.38%
Test Loss: 0.0000, Test Accuracy: 16.99%
Epoch [5/5], Loss: 3.1195, Accuracy: 18.83%
Test Loss: 0.0000, Test Accuracy: 20.00%
metric={'acc_node_0': 25.424107142857142, 'acc_node_1': 20.0, 'layer_name': 'key=resid_post n=4'}


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/5], Loss: 6.2105, Accuracy: 14.92%
Test Loss: 0.0000, Test Accuracy: 17.78%
Epoch [2/5], Loss: 3.1170, Accuracy: 20.61%
Test Loss: 0.0000, Test Accuracy: 20.34%
Epoch [3/5], Loss: 3.0253, Accuracy: 22.44%
Test Loss: 0.0000, Test Accuracy: 24.67%
Epoch [4/5], Loss: 2.9426, Accuracy: 23.49%
Test Loss: 0.0000, Test Accuracy: 23.49%
Epoch [5/5], Loss: 2.8572, Accuracy: 24.15%
Test Loss: 0.0000, Test Accuracy: 23.59%
Epoch [1/5], Loss: 6.8551, Accuracy: 10.95%
Test Loss: 0.0000, Test Accuracy: 14.74%
Epoch [2/5], Loss: 3.3767, Accuracy: 15.92%
Test Loss: 0.0000, Test Accuracy: 17.67%
Epoch [3/5], Loss: 3.2254, Accuracy: 17.50%
Test Loss: 0.0000, Test Accuracy: 19.52%
Epoch [4/5], Loss: 3.1253, Accuracy: 18.44%
Test Loss: 0.0000, Test Accuracy: 17.07%
Epoch [5/5], Loss: 3.1407, Accuracy: 19.16%
Test Loss: 0.0000, Test Accuracy: 20.35%
metric={'acc_node_0': 23.59375, 'acc_node_1': 20.345982142857142, 'layer_name': 'key=resid_post n=5'}


In [83]:
pd.DataFrame(accuracies)

,acc_node_0,acc_node_1,layer_name
0,6.289062,6.227679,key=embed n=None
1,15.625000,12.142857,key=resid_post n=0
2,11.289062,8.950893,key=resid_post n=1
3,18.582589,18.922991,key=resid_post n=2
4,26.858259,20.033482,key=resid_post n=3
5,25.424107,20.000000,key=resid_post n=4
6,23.593750,20.345982,key=resid_post n=5


In [ ]:
# model.fit(X_train[:], y_train[:])